In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

In [2]:
class Threshold_pc:
    def __init__(self, data):
        self.data = data
        self.begin_time = data['time'].min()
        self.end_time = data['time'].max()

    def begin_time(self):
        return self.begin_time

    def end_time(self):
        return self.end_time

    def _calculate(self, time, threshold):
        #print(time)
        values = self.data.loc[(self.data['time'] >= time) & (self.data['time'] < time + timedelta(days=1))]
        #print(values)
        
        tn = values.min()
        tx = values.max()
        tmean = values.mean()
        tmedian = values.median()
        
        # generate a dataframe containing min, max, mean and median
        # drop first row with iloc
        result = pd.concat([tn,tx,tmean,tmedian], axis=1).iloc[1:, :]
        result = result.rename(columns={0:'min', 1:'max', 2:'mean', 3:'median'})

        # Accessing all the values less than threshold 
        # counting them and return % value
        # count number of values and divide by the number of rows
        no_under_threshold = result.loc[result['min'] <= threshold, 'min'].count()
        no_total = result['min'].count()
        pc_tn = no_under_threshold / no_total *100
        return [f"{time:%Y-%m-%d}", f"{no_under_threshold}/{no_total}", f"{pc_tn:.0f} %"]
        #return f"{time:%Y-%m-%d}: {no_under_threshold}/{no_total} = {pc_tn:.0f} %"

    def pc(self, threshold):
        time = self.begin_time
        #self._calculate(time, threshold)
        result = []
        while time < self.end_time:
            result.append(self._calculate(time, threshold))
            time = time + timedelta(days=1)
            #print(time)
        return result

In [3]:
pd.set_option('display.float_format', '{:.1f}'.format)

In [4]:
#url to retrive data from open-meteo
url = 'https://ensemble-api.open-meteo.com/v1/ensemble?latitude=46.2022&longitude=6.1457&hourly=temperature_2m&timezone=Europe%2FBerlin&forecast_days=9&models=icon_seamless'

In [5]:
#retrive data
r = requests.get(url)

In [6]:
t2m = r.json()
#t2m

In [7]:
t2m_hourly_as_list = pd.json_normalize(t2m['hourly'])
#rename temperature_2m as temperature_2m_member00
t2m_hourly_as_list.rename(columns={"temperature_2m": "temperature_2m_member00"}, inplace=True)
#t2m_hourly_as_list

In [14]:
t2m_headers = []
for item in t2m_hourly_as_list.items():
    #print(item[0])
    t2m_headers.append(item[0])
#t2m_headers

In [15]:
t2m_hourly = t2m_hourly_as_list.explode(t2m_headers)
 #t2m_hourly

In [16]:
t2m_hourly['time'] = pd.to_datetime(t2m_hourly['time'], format='%Y-%m-%dT%H:%M')

In [17]:
t2m = Threshold_pc(t2m_hourly)

In [18]:
t2m.pc(0)

[['2024-02-07', '11/40', '28 %'],
 ['2024-02-08', '0/40', '0 %'],
 ['2024-02-09', '0/40', '0 %'],
 ['2024-02-10', '0/40', '0 %'],
 ['2024-02-11', '0/40', '0 %'],
 ['2024-02-12', '0/40', '0 %'],
 ['2024-02-13', '0/40', '0 %'],
 ['2024-02-14', '1/40', '2 %'],
 ['2024-02-15', '2/40', '5 %']]